In [2]:
import numpy as np
import pandas as pd
import os 
import glob
from multiprocessing import Pool

In [3]:
#### Load compactors into a list. 
sponge_cr = pd.read_csv('/oak/stanford/groups/horence/george/protein_domain_project/sponge_compactors_with_repeat_calls.tsv',sep='\t')
sponge_cr = sponge_cr[~sponge_cr['dataset'].str.contains('all_samples')]
arrr = sponge_cr['compactor'].unique()

In [ ]:
#### Write compactors to a FASTA. 
os.mkdir('sponge_pfam')
file = open('sponge_pfam/compactors.fasta','a')
for i in range(len(arrr)):
    file.write('>'+str(i)+'\n'+arrr[i]+'\n')
file.close()
os.chdir('sponge_pfam')

#### Run the Pfam script on the compactors. 
os.system('sbatch /oak/stanford/groups/horence/george/splash_utils/pfam_multithread.sh compactors.fasta')

In [5]:
#### Load files whose anchors we have compactors for.
## Define fields we want. 
usec=['anchor','effect_size_bin',
 'number_nonzero_samples',
 'target_entropy',
 'avg_edit_distance_max_target']
sout = glob.glob('/oak/stanford/groups/horence/Roozbeh/Sponge_project/runs/10X_runs/Sponge_SRP216435/*/result.after_correction.scores.tsv')
fir = pd.read_csv(sout[0],usecols=usec,sep='\t')
fir['dataset'] = sout[0]
for i in range(1,len(sout)):
    nex = pd.read_csv(sout[i],sep='\t',usecols=usec)
    nex['dataset'] = sout[i]
    fir = pd.concat([fir,nex])

In [8]:
with_stats = sponge_cr.merge(fir,how='left')

In [11]:
pfam_files = glob.glob('sponge_pfam/*fasta')

pfams = pd.read_csv(pfam_files[0],engine='python',sep='\t',header=None)
for i in range(1,len(pfam_files)):
    try:
        pfams_1 = pd.read_csv(pfam_files[i],engine='python',sep='\t',header=None)
        pfams = pd.concat([pfams_1,pfams])
    except pd.errors.EmptyDataError:
        pass

pfams = pd.DataFrame({'header':[i for i in pfams[0] if i[0]=='>'],'sequence':[i for i in pfams[0] if i[0]!='>']})

pfams['header'] = [i[1:] for i in pfams['header'] ]


pfam_files = glob.glob('sponge_pfam/*PFAM.tblout')

pfams1 = pd.read_csv(pfam_files[0],engine='python',sep='\t',header=None)
for i in range(1,len(pfam_files)):
    try:
        pfams_1 = pd.read_csv(pfam_files[i],engine='python',sep='\t',header=None)
        pfams1 = pd.concat([pfams_1,pfams1])
    except pd.errors.EmptyDataError:
        pass
    
## Define the fields in the Pfam output. 
fields = ['header','accession','tlen','query_name','accession2','qlen','full_seq_evalue','full_seq_score','full_seq_bias','this_domain_number','this_domain_of','this_domain_c_evalue','this_domain_i_evalue','this_domain_score','this_domain_bias','hmm_coord_from','hmm_coord_to','ali_coord_from','ali_coord_to','env_coord_from','env_coord_to','acc','description_of_target']

## Parse the entries in the Pfam table which are not related to formatting. 
pfam_list_compactor = [i for i in list(pfams1.iloc[:,0]) if '#' not in i]
compactor_ok = [ i.split(' ') for i in pfam_list_compactor]
compactor_okok = []
for i in compactor_ok: 
    lis = [j for j in i if j]
    lis = lis[:22] + [' '.join(lis[23:])]
    compactor_okok.append(lis)

compactor_pfam_structured = pd.DataFrame(compactor_okok, columns = fields)
compactor_pfam_structured = compactor_pfam_structured.add_prefix('Pfam_') 

## As the table has been loaded as a string, convert the numerical field (e-value) to a float to ensure 
## integrity of operations using this value. 
compactor_pfam_structured['Pfam_full_seq_evalue'] = compactor_pfam_structured['Pfam_full_seq_evalue'].astype(float)

## Filter so that we retain e-values < 0.05. 
compactor_pfam_structured = compactor_pfam_structured[compactor_pfam_structured['Pfam_full_seq_evalue']<0.05].reset_index(drop=True)

compactor_pfam_structured = compactor_pfam_structured.rename(columns={'Pfam_header':'header'})

compactor_pfam_structured['Pfam_frame'] = [int(i.split('=')[1]) for i in compactor_pfam_structured['header']]
compactor_pfam_structured['Pfam_strand'] = [i>0 for i in compactor_pfam_structured['Pfam_frame']]

compactor_pfam_structured[['Pfam_env_coord_from','Pfam_env_coord_to']] = compactor_pfam_structured[['Pfam_env_coord_from','Pfam_env_coord_to']].astype(int)
compactor_pfam_structured[['Pfam_this_domain_i_evalue']] = compactor_pfam_structured[['Pfam_this_domain_i_evalue']].astype(float)
compactor_pfam_structured['Pfam_strand'] = compactor_pfam_structured['Pfam_strand'].replace({True:'+',False:'-'})

## Get a Pfam results file. 
compactor_pfam_structured['header'] = compactor_pfam_structured['header'].str.split('_').str[0]
pfam_hits = compactor_pfam_structured.merge(pfams,how='left')



In [13]:
def inds(data):
    
    ## Create a local copy. 
    copy = data.sort_values('Pfam_this_domain_i_evalue')

    ## Get the index corresponding to the best e-value hit. 
    take_index = [int(copy.index[0])]
    
    ## Get the range of seqeunce coordinates corresponding to the hit. 
    initial_coord_set = set(list(range(int(copy['Pfam_env_coord_from'][take_index[0]]),int(copy['Pfam_env_coord_to'][take_index[0]]+1))))
    
    ## For each remaining hit: 
    for ind in copy.index[1:]:
        
        ## Extrac the range of sequence coordinates. 
        this_coord_set = set(list(range(int(copy['Pfam_env_coord_from'][ind]),int(copy['Pfam_env_coord_to'][ind])+1)))
        
        ## Get the intersect of this hit's sequence coordinates and all accepted hits' coordinates. 
        intersect = set(this_coord_set&initial_coord_set)
        
        ## If the intersect size is 0, we do not have a best hit for these coordinates, so we add this one.
        ## Further, we update the set of coordinates for which we have best hits. 
        if len(intersect) == 0:
            take_index.append(ind)
            initial_coord_set = set(this_coord_set|initial_coord_set)

    ## We return the indices corresponding to best hits covering the input sequence. 
    return take_index

def applyParallel(dfGrouped, func):
    with Pool(int(os.environ['SLURM_JOB_CPUS_PER_NODE'])) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])
    return ret_list

def flatten_extend(matrix):
    """
    https://realpython.com/python-flatten-list/
    """
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [17]:
#### Get nonoverlapping hits. 
gpby = pfam_hits.groupby(['header','Pfam_strand'])[['Pfam_this_domain_i_evalue','Pfam_env_coord_from','Pfam_env_coord_to']]
outs = applyParallel(gpby, inds)
best_spots = pfam_hits.loc[flatten_extend(outs)]
best_spots = best_spots.rename(columns={'sequence':'compactor'})

In [18]:
## A file reporting compactors, their SPLASH statistics, all (nonoverlapping) Pfam hits. 
stats_and_all_hits = with_stats.merge(best_spots,how='left')